In [129]:
import os
import xml.etree.ElementTree as ET

In [130]:
import sqlite3

In [131]:
# path on rte
db_path = "/home/rte/data/db/arxiv_db_images.sqlite3"

In [132]:
db = sqlite3.connect(db_path)
c = db.cursor()

In [133]:
c.execute('PRAGMA TABLE_INFO({})'.format("metadata"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)


Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'identifier', 'TEXT', 0, None, 0)
(2, 'created', 'TEXT', 0, None, 0)
(3, 'cat', 'TEXT', 0, None, 0)
(4, 'authors', 'TEXT', 0, None, 0)
(5, 'title', 'TEXT', 0, None, 0)
(6, 'abstract', 'TEXT', 0, None, 0)
(7, 'licence', 'TEXT', 0, None, 0)


In [134]:
c.execute('PRAGMA TABLE_INFO({})'.format("images"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)


Column Info:
ID, Name, Type, NotNull, DefaultVal, PrimaryKey
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'identifier', 'TEXT', 0, None, 0)
(2, 'filename', 'TEXT', 0, None, 0)
(3, 'filesize', 'INT', 0, None, 0)
(4, 'path', 'TEXT', 0, None, 0)
(5, 'x', 'INT', 0, None, 0)
(6, 'y', 'INT', 0, None, 0)
(7, 'imageformat', 'TEXT', 0, None, 0)


In [135]:
# Find where there is no matching metadata for a given image

c.execute('''
    SELECT count(images.identifier), images.identifier
    FROM images
    LEFT JOIN metadata ON metadata.identifier = images.identifier
    WHERE metadata.identifier IS NULL
    GROUP BY images.identifier
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

(4, 'acc-phys9607002')
(1, 'math0402448')
(12, 'math0609138')
(17, 'nucl-th0501065')


In [83]:
print(sum(row[0] for row in rows))

34


In [84]:
# check which articles don't have a matching file

os.chdir("/home/rte/arXiv/oai/")

for filename in (row[1] for row in rows):

    if os.path.isfile("oai:arXiv.org:" + filename + ".arXiv.xml") is False:
        print(filename)
#         print("no matching file")

acc-phys9607002
math0402448
math0609138
nucl-th0501065


In [88]:
filenames = ["math%2F0402448", "math%2F0609138", "nucl-th%2F0501065"]
# "acc-phys9607002"    

In [96]:
os.chdir("/home/rte/arXiv/oai/")
# os.chdir("/home/rte/data/extra-metadata")

OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"

# filenames = ["0804.3168"]

# for filename in (row[1] for row in rows):
for filename in filenames:
    filepath = "oai:arXiv.org:" + filename + ".arXiv.xml"
#     filepath = filename

    if os.path.isfile(filepath) is False:
        print(filename + " does not exist at this lcoation")
    else:
        data = ET.parse(filepath)
        print("opening file: " + filepath)
        root = data.getroot()    
        
        try:
            
#             for child in root:
#                 print(child.tag, child.attrib)
    
            id_node = root.find(ARXIV+'id')
#             print(id_node)
            identifier = id_node.text
#             print(identifier)
            
            date = root.find(ARXIV+'created').text
            categories = root.find(ARXIV+'categories').text
            title = root.find(ARXIV+'title').text
            abstract = root.find(ARXIV+'abstract').text
            
            license_node = root.find(ARXIV+"license")

            if license_node is not None:
                lic = license_node.text
                # print("license_node not None")
            else:
                lic = ""
            
            # attempt to get authors, not working, needs work
            authors = []
            authors_element = root.find(ARXIV+"authors")
            
            # create a variable to store all authors names
            anames = ""

            for author in authors_element:
                # print(author.find(ARXIV+"keyname").text)
                aname = ""
                kn = author.find(ARXIV+"keyname").text
                fn_node = author.find(ARXIV+"forenames")
                if fn_node is not None:
                    fn = fn_node.text
                else:
                    fn = ""
                aname =  kn + ", " + fn + "; "

                # for fn in authors_element.find(ARXIV+"forenames"):
                    # aname += fn
                    # print(fn)
                # + author.find(ARXIV+"forenames").text
                # print(aname)

                anames += aname
                # for child in author:
                    # print(child.tag, child.attrib)
                # authors.append(author.find("keyname").text)
                # authors.append(author.find("forenames").text)

            authors.append(anames)
            
            print("-" * 20)
            print(identifier)
            print(date)
            print(categories)
#             print(authors)
            authors = "" + str(authors)
            print(authors)
            print(title)
            print(abstract)
            print(lic) # don't use license as it is reserved for Python!
            print("-" * 20)
            
            c.execute("INSERT INTO metadata (identifier, created, cat, authors, title, abstract, licence) \
            VALUES (?, ?, ?, ?, ?, ?, ?)", \
            (identifier, date, categories, authors, title, abstract, lic))
            # , authors, title, abstract, licence
            # , '{3}', '{4}', '{5}', '{6}'
#             c.close()
            
        except KeyboardInterrupt:
            db.commit()

            # quit
            sys.exit()
        # except AttributeError as error:
            # print(error)
            # continue
        except Exception as e:
            raise e
        

ParseError: unbound prefix: line 1, column 0 (<string>)

In [136]:
filenames = ["acc-phys9607002"]

In [143]:
# for last single xml file

bVerbose = True

os.chdir("/home/rte/data/extra-metadata/")

OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"

# for filename in (row[1] for row in rows):
for filename in filenames:
#     filepath = "oai:arXiv.org:" + filename + ".arXiv.xml"
    filepath = filename + ".xml"
    print(filepath)

    data = ET.parse(filepath)
    if(bVerbose):
        print("opening file: " + filepath)
    root = data.getroot()
    
    for child in root:
        print(child.tag, child.attrib)

    for GetRecord in root.findall(ARXIV+'GetRecord'):
        for record in GetRecord.findall(ARXIV+'record'):
            print("record?")
        
            for child in record:
                print(child.tag, child.attrib)
            
            try:
                # arxiv_id = record.find('header').find('identifier')

                meta = record.find(ARXIV+'metadata')
                print(meta)
                # this gets the arXiv element and then gets the text
                # for some reason it grabs the identifier : /
                id_node = meta.find(ARXIV+"arXiv/")
                if id_node is not None:
                    identifier = id_node.text
                else:
                    print("CONTINUING PAST RECORD - NO ID")
                    continue

                info = meta.find(ARXIV+"arXiv")
                print(info)
                date = info.find(ARXIV+"created").text
                # date = datetime.datetime.strptime(date, "%Y-%m-%d")
                categories = info.find(ARXIV+"categories").text
                title = info.find(ARXIV+"title").text
                abstract = info.find(ARXIV+"abstract").text

                license_node = info.find(ARXIV+"license")

                if license_node is not None:
                    lic = license_node.text
                    # print("license_node not None")
                else:
                    lic = ""

                # attempt to get authors, not working, needs work
                authors = []
                authors_element = info.find(ARXIV+"authors")

                # create a variable to store all authors names
                anames = ""

                for author in authors_element:
                    # print(author.find(ARXIV+"keyname").text)
                    aname = ""
                    kn = author.find(ARXIV+"keyname").text
                    fn_node = author.find(ARXIV+"forenames")
                    if fn_node is not None:
                        fn = fn_node.text
                    else:
                        fn = ""
                    aname =  kn + ", " + fn + "; "

                    # for fn in authors_element.find(ARXIV+"forenames"):
                        # aname += fn
                        # print(fn)
                    # + author.find(ARXIV+"forenames").text
                    # print(aname)

                    anames += aname
                    # for child in author:
                        # print(child.tag, child.attrib)
                    # authors.append(author.find("keyname").text)
                    # authors.append(author.find("forenames").text)

                authors.append(anames)

                if(bVerbose):
                    print("-" * 20)
                    print(identifier)
                    print(date)
                    print(categories)
                    authors = "" + str(authors)
                    print(authors)
                    print(title)
                    print(abstract)
                    print(lic) # don't use license as it is reserved for Python!

    #             c = db.cursor()

                c.execute("INSERT INTO metadata (identifier, created, cat, authors, title, abstract, licence) \
                VALUES (?, ?, ?, ?, ?, ?, ?)", \
                (identifier, date, categories, authors, title, abstract, lic))

            except KeyboardInterrupt:
                db.commit()

                # quit
                sys.exit()
            # except AttributeError as error:
                # print(error)
                # continue
            except Exception as e:
                raise e
            # finally:

acc-phys9607002.xml
opening file: acc-phys9607002.xml
{http://arxiv.org/OAI/arXiv/}responseDate {}
{http://arxiv.org/OAI/arXiv/}request {'verb': 'GetRecord', 'identifier': 'oai:arXiv.org:acc-phys/9607002', 'metadataPrefix': 'arXiv'}
{http://arxiv.org/OAI/arXiv/}GetRecord {}
record?
{http://arxiv.org/OAI/arXiv/}header {}
{http://arxiv.org/OAI/arXiv/}metadata {}
<Element '{http://arxiv.org/OAI/arXiv/}metadata' at 0x7ff1316becc8>
<Element '{http://arxiv.org/OAI/arXiv/}arXiv' at 0x7ff1316bed18>
--------------------
acc-phys/9607002
1996-07-11
acc-phys physics.acc-ph
['Kurennoy, Sergey S.; ']
Impedances and Power Losses for an Off-Axis Beam
  A method for calculating coupling impedances and power losses for off-axis
beams is developed. It is applied to calculate impedances of small localized
discontinuities like holes and slots, as well as the impedance due to a finite
resistivity of chamber walls, in homogeneous chambers with an arbitrary shape
of the chamber cross section. The approach re

In [ ]:
# semi-manual entry for last entry

identifier = "acc-phys/9607002"
date = "1996-07-11"

            
c.execute("INSERT INTO metadata (identifier, created, cat, authors, title, abstract, licence) \
VALUES (?, ?, ?, ?, ?, ?, ?)", \
(identifier, date, categories, authors, title, abstract, lic))
# , authors, title, abstract, licence
# , '{3}', '{4}', '{5}', '{6}'
#             c.close()

In [144]:
db.commit()

In [145]:
c.close()
db.close()

In [ ]:
# save (commit) changes

db.commit()